In [1]:
%pip install blimpy
%pip install hdf5plugin
%pip install h5py
%pip install torchvision
%pip install transformers
%pip install torch
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
# import time
# import os
# import random
# import psutil
# import ultralytics
from ultralytics import YOLO
import torch
%matplotlib inline

In [ ]:
import os
from PIL import Image
from collections import Counter

# folder = '/datax/scratch/jliang/small_dataset/train/images'  # change this to your image folder
# sizes = []

# for fname in os.listdir(folder):
#     if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
#         path = os.path.join(folder, fname)
#         with Image.open(path) as img:
#             sizes.append(img.size)  # (width, height)

# # Print top image sizes
# count = Counter(sizes)
# for size, freq in count.most_common(10):
#     print(f"Size {size}: {freq} images")

Size (828, 625): 498 images
Size (781, 524): 229 images


In [ ]:
import os
from PIL import Image
from collections import Counter

# folder = '/datax/scratch/jliang/small_dataset/val/images'  # change this to your image folder
# sizes = []

# for fname in os.listdir(folder):
#     if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
#         path = os.path.join(folder, fname)
#         with Image.open(path) as img:
#             sizes.append(img.size)  # (width, height)

# # Print top image sizes
# count = Counter(sizes)
# for size, freq in count.most_common(10):
#     print(f"Size {size}: {freq} images")

Size (828, 625): 115 images
Size (781, 524): 65 images


In [ ]:
# # Check if the dataset YAML is valid using Ultralytics utility
# from ultralytics.utils import checks

# yaml_path = '/datax/scratch/jliang/small_dataset/data.yaml'
# try:
# 	checked_yaml = checks.check_yaml(yaml_path)
# 	print(f"YAML file is valid: {checked_yaml}")
# except Exception as e:
# 	print(f"Error in YAML file: {e}")

YAML file is valid: /datax/scratch/jliang/small_dataset/data.yaml


In [ ]:
import torch
from ultralytics import YOLO

# Start at 16 and automatically back off on OOM
batch_size = 16
model = YOLO("yolo11n.pt")

# --- Custom hyperparameter tweaks from earlier advice ---
# 1) Halve the cls/DFL loss weights
# 2) Slow the ramp to full weight to 10 epochs
# 3) (Optional) you could load defaults from a hyp.yaml and then modify
custom_hyp = {
    "cls": 0.5,           # reduce classification loss weight
    "dfl": 0.5,           # reduce distribution focal loss weight
    "warmup_epochs": 10,  # ramp cls/DFL up over 10 epochs instead of 4–5
}

while True:
    try:
        model.train(
            data="/home/jliang/gbt-rfi/data.yaml",
            epochs=100,
            batch=batch_size,
            workers=2,

            # — Lower initial LR so you don’t explode when cls/DFL hit full weight —
            lr0=0.005,

            # — Override default hyp params —
            hyp=custom_hyp,

            # — Gradient clipping to guard against spikes —
            clip=10.0,

            # — Recompute anchors on your spectrogram boxes —
            autoanchor=True,

            # — Accept smaller IoU as “positive” to match tiny events —
            iou_t=0.3,

            # — Keep your other augment/settings choices —
            mosaic=False,
            rect=True,
            fliplr=0,
            patience=10,

            name="yolo-moe-augmented-v2",
        )
        print(f"✔️  Training started successfully with batch_size={batch_size}")
        break

    except (RuntimeError, torch.cuda.OutOfMemoryError) as e:
        msg = str(e).lower()
        if "out of memory" in msg:
            if batch_size == 1:
                print("[OOM] Even batch_size=1 OOMs—aborting.")
                break
            new_bs = batch_size // 2
            print(f"[OOM] batch_size={batch_size} too large, dropping to {new_bs}...")
            batch_size = new_bs
            torch.cuda.empty_cache()
        else:
            # re-raise anything unexpected
            raise


In [ ]:
# batch_size = 16

# while batch_size >= 1:
#     try:
#         model = YOLO("yolo11n.pt")
#         model.train(
#             data='/home/jliang/gbt-rfi/data.yaml',
#             epochs=100,
#             mosaic=False,
#             rect=True,
#             fliplr=0,
#             batch=batch_size,
#             name='yolo-moe-augmented',
#             workers=2,  # Lower this if you're seeing CPU/memory load issues
#             patience=10
#         )
#         break
#     except RuntimeError as e:
#         if 'Out of Memory' in str(e):
#             print(f"[OOM] Batch size {batch_size} too large. Trying {batch_size // 2}...")
#             torch.cuda.empty_cache()
#             batch_size = batch_size // 2
#         else:
#             raise



New https://pypi.org/project/ultralytics/8.3.159 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.158 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA RTX A4000, 16102MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/datax/scratch/jliang/small_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo-attempt2-small, nbs=64

train: Scanning /datax/scratch/jliang/small_dataset/train/labels... 12 images, 0 backgrounds, 0 corrupt: 100%|██████████| 12/12 [00:00<00:00, 215.41it/s]

train: New cache created: /datax/scratch/jliang/small_dataset/train/labels.cache
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA RTX A4000) 15.72G total, 0.19G reserved, 0.14G allocated, 15.40G free


      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     2590035       6.441         0.474         487.9          8341        (1, 3, 640, 640)                    list
     2590035       12.88         0.659         32.87         476.1        (2, 3, 640, 640)                    list
     2590035       25.76         0.973         33.29         265.9        (4, 3, 640, 640)                    list
     2590035       51.52         1.657         62.17         287.4        (8, 3, 640, 640)                    list
     2590035         103         2.913         42.31         388.5       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 53 for CUDA:0 9.26G/15.72G (59%) ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 99.0±52.1 MB/s, size: 44.4 KB)


train: Scanning /datax/scratch/jliang/small_dataset/train/labels... 13 images, 0 backgrounds, 0 corrupt: 100%|██████████| 13/13 [00:00<00:00, 1776.44it/s]

...

                   all          3        110          0          0          0          0



50 epochs completed in 0.017 hours.
Optimizer stripped from runs/detect/yolo-attempt2-small/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/yolo-attempt2-small/weights/best.pt, 5.5MB

Validating runs/detect/yolo-attempt2-small/weights/best.pt...
Ultralytics 8.3.158 🚀 Python-3.12.3 torch-2.7.1+cu126 CUDA:0 (NVIDIA RTX A4000, 16102MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 29.03it/s]


                   all          3        110          0          0          0          0
Speed: 0.2ms preprocess, 4.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/yolo-attempt2-small


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x76979e22f770>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
import os
from pathlib import Path

import cv2
from PIL import Image, ImageDraw
from ultralytics import YOLO  # ← make sure you have ultralytics installed

# 1) Configuration — adjust paths & model name
VAL_IMG_DIR   = Path("/datax/scratch/jliang/augmented/val/images")
VAL_LABEL_DIR = Path("/datax/scratch/jliang/augmented/val/labels")
OUTPUT_DIR    = Path("/datax/scratch/jliang/yolo_vis_output")
MODEL_WEIGHTS = "/home/jliang/gbt-rfi/runs/detect/yolo-moe-v1/weights/best.pt"
CONF_THRESH   = 0.25

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 2) Load the YOLOv11 model
model = YOLO(MODEL_WEIGHTS)
model.conf = CONF_THRESH

def load_yolo_labels(label_path):
    boxes = []
    with open(label_path, "r") as f:
        for line in f:
            cls, xc, yc, w, h = line.strip().split()
            boxes.append((int(cls), float(xc), float(yc), float(w), float(h)))
    return boxes

def draw_boxes(img, boxes, color, width=2):
    W, H = img.size
    draw = ImageDraw.Draw(img)
    for xc, yc, w, h in boxes:
        x0 = (xc - w/2) * W
        x1 = (xc + w/2) * W
        y0 = (yc - h/2) * H
        y1 = (yc + h/2) * H
        draw.rectangle([x0, y0, x1, y1], outline=color, width=width)

# 3) Loop over all validation images
for img_path in VAL_IMG_DIR.glob("*.*"):  # .png, .jpg, etc.
    # a) load image
    img = Image.open(img_path).convert("RGB")

    # b) load GT
    label_path = VAL_LABEL_DIR / f"{img_path.stem}.txt"
    gt = load_yolo_labels(label_path)
    gt_boxes = [(xc, yc, w, h) for (_, xc, yc, w, h) in gt]

    # c) run inference
    results = model(str(img_path))

    # d) extract YOLOv8-style boxes (xyxy, conf, cls)
    det = results[0].boxes
    xyxy   = det.xyxy.cpu().numpy()   # shape [N,4]
    confs  = det.conf.cpu().numpy()   # shape [N]
    cls_ids= det.cls.cpu().numpy().astype(int)  # shape [N]

    # e) convert to normalized (xc,yc,w,h)
    pred_boxes = []
    w_img, h_img = img.width, img.height
    for (x1, y1, x2, y2), conf, cls in zip(xyxy, confs, cls_ids):
        if conf < CONF_THRESH:
            continue
        pw = x2 - x1
        ph = y2 - y1
        xc = x1 + pw/2
        yc = y1 + ph/2
        # normalize
        xc /= w_img
        yc /= h_img
        pw /= w_img
        ph /= h_img
        pred_boxes.append((xc, yc, pw, ph))

    # f) draw GT in red, predictions in blue
    vis = img.copy()
    draw_boxes(vis, gt_boxes,   color="red",   width=2)
    draw_boxes(vis, pred_boxes, color="blue",  width=2)

    # g) save
    out_path = OUTPUT_DIR / f"{img_path.stem}_vis.png"
    vis.save(out_path)
    print(f"Saved {out_path}")

